### 자연어 감성 분석
- 감성사전 기반 : 미리 저으이된 감성단어 사전 사용(규칙 기반)
    - TextBlob, AFINNm VADER
- 머신러닝 기반 : 데이터로 부터 패턴 학습(통계 기반)
    - TF-IDF 벡터화
    - 빈도수 벡터화
    - Multinomial Naive Bayes
    - 로지스틱 회귀

### 사용 데이터
- NLTK 영화 리뷰 (2000개)
- 다음영화리뷰

#### 알고리즘
- textblob
- afinn
- vader

- TF-IDF
- CounterVectorizer
- Multinomial Naive Bayes
- 로지스틱 회귀 ^

In [ ]:
# TextBlob
# 특정단어를 선정하고, 그 단어가 나올경우 점수를 추가하거나 감점한다.
# Polarity  극성도
# 긍정과 부정
# 0은 중립
# Subjectivity 주관성
# 객관과 주관

# 문맥은 무시하고 단어 여부로만 판단한다.
# 이 영화는 나쁘지 않다. -> 나쁘다, 않다 에서 감점(-)으로 인식
# 장점 : 빠른속도 학습 불필요
# 사용 : 실시간 감성 분석, 스트리밍 데이터 -> 빠른 판단이 필요한 상황에 사용하기도 한다.

In [ ]:
%pip install TextBlob

In [ ]:
from textblob import TextBlob, Word
text = "TextBlob is amazingly simple to use. What a wonderful library for NLP!"
blob = TextBlob(text)
print(blob.sentences)
print(blob.words)
# 기본적인 문장, 단어 분리 기능을 가지고있는 것을 확인할 수 있다.